In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
import dagshub
import mlflow
import mlflow.sklearn

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [3]:
train_transaction = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
train_identity = pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')
df = train_transaction.merge(train_identity, on='TransactionID', how='left')

In [4]:
dagshub.init(repo_owner='gioeba', repo_name='IEEE-CIS-Fraud-Detection', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=faf19b59-2788-4cb2-8098-40972752327b&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=648941763f030810dd455d2ecb3fddf996150401df7dc6179ae80b0277cb2a9a




Output()

Accessing as gioeba

Initialized MLflow to track repo "gioeba/IEEE-CIS-Fraud-Detection"

Repository gioeba/IEEE-CIS-Fraud-Detection initialized!

# Cleaning

In [5]:
y = df['isFraud']
X = df.drop(columns=['isFraud', 'TransactionID']).copy()

# Feature Engineering

In [6]:
START_DATE = pd.to_datetime('2017-12-01')
X['TransactionDT'] = pd.to_timedelta(X['TransactionDT'], unit='s')
X['datetime'] = START_DATE + X['TransactionDT']
X['hour'] = X['datetime'].dt.hour
X['weekday'] = X['datetime'].dt.weekday
X['day'] = X['datetime'].dt.day
X.drop(columns=['datetime'], inplace=True)

X['DeviceInfo'] = X['DeviceInfo'].fillna('unknown').str.lower()
X['device_os'] = X['DeviceInfo'].str.extract(r'([a-z]+)', expand=False)
X['device_ver'] = X['DeviceInfo'].str.extract(r'(\d+)', expand=False)

X = X.loc[:, X.isnull().mean() < 0.5].copy()

if 'card1' in X.columns and 'card2' in X.columns:
    X['card1_card2_ratio'] = X['card1'] / (X['card2'] + 1)

if 'card1' in X.columns and 'addr1' in X.columns:
    X['card1_addr1'] = X['card1'].astype(str) + '_' + X['addr1'].astype(str)

if 'card1' in X.columns and 'TransactionAmt' in X.columns:
    X['card1_count'] = X.groupby('card1')['TransactionAmt'].transform('count')
    X['card1_mean'] = X.groupby('card1')['TransactionAmt'].transform('mean')

# Feature Selection

In [7]:
cat_cols = X.select_dtypes('object').columns.tolist()
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

num_transform = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

cat_transform = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

preprocessor = ColumnTransformer([
    ('num', num_transform, num_cols),
    ('cat', cat_transform, cat_cols)
])

clf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(max_depth=10, class_weight='balanced', random_state=42))
])

# Training

In [8]:
mlflow.set_experiment("ieee-decision-tree")
with mlflow.start_run(run_name="tree-depth-10-fe"):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    auc_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(df, y), 1):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
        clf_pipeline.fit(X_train, y_train)
        y_prob = clf_pipeline.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, y_prob)
        auc_scores.append(auc)
    
        print(f"Fold {fold} AUC: {auc:.4f}")
        mlflow.log_metric(f"fold_{fold}_auc", auc)
    
    avg_auc = np.mean(auc_scores)
    mlflow.log_metric("avg_roc_auc", avg_auc)
    mlflow.log_param("model", "DecisionTree")
    mlflow.log_param("max_depth", 10)
    mlflow.sklearn.log_model(clf_pipeline, "model")
    print(f"\nAverage AUC across folds: {avg_auc:.4f}")

Fold 1 AUC: 0.8712
Fold 2 AUC: 0.8736
Fold 3 AUC: 0.8757
Fold 4 AUC: 0.8633
Fold 5 AUC: 0.8642


2025/04/21 16:59:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Average AUC across folds: 0.8696
🏃 View run tree-depth-10-fe at: https://dagshub.com/gioeba/IEEE-CIS-Fraud-Detection.mlflow/#/experiments/0/runs/dd3633d1b39a4db5a1c704f4c1a1a970
🧪 View experiment at: https://dagshub.com/gioeba/IEEE-CIS-Fraud-Detection.mlflow/#/experiments/0
